### Code for parsers

In [1]:
import re

In [8]:
lines = [
    '# Groceries',
    '; account equity',
    '2024-02-02 paycheck',
    '2024-01-07  Safeway ; weekly groceries',
    '    income:yoyodyne      $5,000.00',
    '    income:yoyodyne      $5,000.00  ; Feb',
]

In [9]:
for s in lines:
    cmnd, *comment = re.split(r'[;#]', s)
    print(cmnd, comment)


 [' Groceries']
 [' account equity']
2024-02-02 paycheck []
2024-01-07  Safeway  [' weekly groceries']
    income:yoyodyne      $5,000.00 []
    income:yoyodyne      $5,000.00   [' Feb']


In [4]:
amounts = [
    '123.45',
    '$123.45',
    '-123.45',
    '$-123.45',
    '-$123.45',
    '9,123.45',
    '$9,123.45',
    '-9,123.45',
    '$-9,123.45',
    '-$9,123.45',    
]

In [6]:
for s in amounts:
    t = re.sub(r'[,$]','',s)
    print(float(t))


123.45
123.45
-123.45
-123.45
-123.45
9123.45
9123.45
-9123.45
-9123.45
-9123.45
